In [0]:
import keras
keras.__version__

Using TensorFlow backend.


'2.2.4'

## Recurrent neural networks for weather prediction

This notebook adapts the code samples found in Chapter 6, Section 3 of [Deep Learning with Python](https://www.manning.com/books/deep-learning-with-python?a_aid=keras&a_bid=76564dff). Note that the original text features far more content, in particular further explanations and figures: in this notebook, you will only find source code and related comments.

---

We use a GRU together with

* *Recurrent dropout*, a specific, built-in way to use dropout to fight overfitting in recurrent layers.

In Model 1, we add a skip connection that adds the baseline nonlearning prediction.

In Model 2, we add encoded time and date.  

In [0]:
model_num = 1 # 2

## A temperature forecasting problem

Until now, the only sequence data we have covered has been text data, for instance the IMDB dataset and the Reuters dataset. But sequence 
data is found in many more problems than just language processing. In all of our examples in this section, we will be playing with a weather 
timeseries dataset recorded at the Weather Station at the Max-Planck-Institute for Biogeochemistry in Jena, Germany: http://www.bgc-jena.mpg.de/wetter/.

In this dataset, fourteen different quantities (such air temperature, atmospheric pressure, humidity, wind direction, etc.) are recorded 
every ten minutes, over several years. The original data goes back to 2003, but we limit ourselves to data from 2009-2016. This dataset is 
perfect for learning to work with numerical timeseries. We will use it to build a model that takes as input some data from the recent past (a 
few days worth of data points) and predicts the air temperature 24 hours in the future.

Let's take a look at the data:

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
import os

data_dir = '/content/gdrive/My Drive/Datasets'
fname = os.path.join(data_dir, 'jena_climate_2009_2016.csv')

f = open(fname)
data = f.read()
f.close()

lines = data.split('\n')
header = lines[0].split(',')
lines = lines[1:]

print(header)
print(len(lines))

['"Date Time"', '"p (mbar)"', '"T (degC)"', '"Tpot (K)"', '"Tdew (degC)"', '"rh (%)"', '"VPmax (mbar)"', '"VPact (mbar)"', '"VPdef (mbar)"', '"sh (g/kg)"', '"H2OC (mmol/mol)"', '"rho (g/m**3)"', '"wv (m/s)"', '"max. wv (m/s)"', '"wd (deg)"']
420551


In [0]:
print(lines[1000:1003])

['07.01.2009 22:50:00,997.79,-10.79,262.54,-11.84,91.90,2.68,2.46,0.22,1.53,2.47,1323.57,0.24,0.63,139.40', '07.01.2009 23:00:00,997.88,-10.68,262.64,-11.74,91.80,2.70,2.48,0.22,1.55,2.48,1323.13,0.10,0.50,184.20', '07.01.2009 23:10:00,998.05,-10.99,262.32,-12.14,91.10,2.63,2.40,0.23,1.50,2.40,1324.97,0.20,0.63,146.60']


Let's convert all of these 420,551 lines of data into a Numpy array:

In [0]:
import numpy as np

month_lens = [0,31,28,31,30,31,30,31,31,30,31,30]
cum_len = np.cumsum(month_lens)

float_data = np.zeros((len(lines), len(header) - 1))
time_codes = np.zeros((len(lines), 4))

for i, line in enumerate(lines):
    # separate the entries and convert all but first to float
    entries = line.split(',')
    values = [float(x) for x in entries[1:]]
    float_data[i, :] = values
    
    # Get the month and date and convert to unit circle
    date = float(entries[0][0:2])
    month = int(entries[0][3:5])
    year_frac = (date + cum_len[month-1])/365.0
    time_codes[i,0] = np.sin(year_frac)
    time_codes[i,1] = np.cos(year_frac)
    
    # Get the hour and minute and convert to unit circle
    hour = float(entries[0][11:13])
    minute = float(entries[0][14:16])
    day_frac = (hour + minute/60.0)/24.0
    time_codes[i,2] = np.sin(day_frac)
    time_codes[i,3] = np.cos(day_frac)



In [0]:
print(float_data[1000:1003])
print(time_codes[1000:1003, :])

[[ 9.97790e+02 -1.07900e+01  2.62540e+02 -1.18400e+01  9.19000e+01
   2.68000e+00  2.46000e+00  2.20000e-01  1.53000e+00  2.47000e+00
   1.32357e+03  2.40000e-01  6.30000e-01  1.39400e+02]
 [ 9.97880e+02 -1.06800e+01  2.62640e+02 -1.17400e+01  9.18000e+01
   2.70000e+00  2.48000e+00  2.20000e-01  1.55000e+00  2.48000e+00
   1.32313e+03  1.00000e-01  5.00000e-01  1.84200e+02]
 [ 9.98050e+02 -1.09900e+01  2.62320e+02 -1.21400e+01  9.11000e+01
   2.63000e+00  2.40000e+00  2.30000e-01  1.50000e+00  2.40000e+00
   1.32497e+03  2.00000e-01  6.30000e-01  1.46600e+02]]
[[0.01917691 0.99981611 0.81422261 0.58055279]
 [0.01917691 0.99981611 0.81823456 0.57488451]
 [0.01917691 0.99981611 0.82220706 0.56918851]]


## Preparing the data


The exact formulation of our problem will be the following: given data going as far back as `lookback` timesteps (a timestep is 10 minutes) 
and sampled every `steps` timesteps, can we predict the temperature in `delay` timesteps?

We will use the following parameter values:

* `lookback = 720`, i.e. our observations will go back 5 days.
* `steps = 6`, i.e. our observations will be sampled at one data point per hour.
* `delay = 144`, i.e. our targets will be 24 hours in the future.

To get started, we need to do two things:

* Preprocess the data to a format a neural network can ingest. This is easy: the data is already numerical, so we don't need to do any 
vectorization. However each timeseries in the data is on a different scale (e.g. temperature is typically between -20 and +30, but 
pressure, measured in mbar, is around 1000). So we will normalize each timeseries independently so that they all take small values on a 
similar scale.
* Write a Python generator that takes our current array of float data and yields batches of data from the recent past, alongside with a 
target temperature in the future. Since the samples in our dataset are highly redundant (e.g. sample `N` and sample `N + 1` will have most 
of their timesteps in common), it would be very wasteful to explicitly allocate every sample. Instead, we will generate the samples on the 
fly using the original data.

We preprocess the data by subtracting the mean of each timeseries and dividing by the standard deviation. We plan on using the first 
200,000 timesteps as training data, so we compute the mean and standard deviation only on this fraction of the data:

In [0]:
mean = float_data[:200000].mean(axis=0)
float_data -= mean
std = float_data[:200000].std(axis=0)
float_data /= std


Now here is the data generator that we will use. It yields a tuple `(samples, targets)` where `samples` is one batch of input data and 
`targets` is the corresponding array of target temperatures. It takes the following arguments:

* `data`: The original array of floating point data, which we just normalized in the code snippet above.
* `lookback`: How many timesteps back should our input data go.
* `delay`: How many timesteps in the future should our target be.
* `min_index` and `max_index`: Indices in the `data` array that delimit which timesteps to draw from. This is useful for keeping a segment 
of the data for validation and another one for testing.
* `shuffle`: Whether to shuffle our samples or draw them in chronological order.
* `batch_size`: The number of samples per batch.
* `step`: The period, in timesteps, at which we sample data. We will set it 6 in order to draw one data point every hour.

In [0]:
def generator(data, lookback, delay, min_index, max_index,
              shuffle=False, batch_size=128, step=6):
    if max_index is None:
        max_index = len(data) - delay - 1
    i = min_index + lookback
    while 1:
        if shuffle:
            rows = np.random.randint(
                min_index + lookback, max_index, size=batch_size)
        else:
            if i + batch_size >= max_index:
                i = min_index + lookback
            rows = np.arange(i, min(i + batch_size, max_index))
            i += len(rows)

        samples = np.zeros((len(rows),
                           lookback // step,
                           data.shape[-1]))
        targets = np.zeros((len(rows),))
        for j, row in enumerate(rows):
            indices = range(rows[j] - lookback, rows[j], step)
            samples[j] = data[indices]
            targets[j] = data[rows[j] + delay][1]
        yield samples, targets


Now let's use our abstract generator function to instantiate three generators, one for training, one for validation and one for testing. 
Each will look at different temporal segments of the original data: the training generator looks at the first 200,000 timesteps, the 
validation generator looks at the following 100,000, and the test generator looks at the remainder.

In [0]:
lookback = 432
step = 6
delay = 144
batch_size = 128

if (model_num == 2):  # add the time codes for Model 2
  float_data = np.concatenate((float_data, time_codes), axis=1)

train_gen = generator(float_data,
                      lookback=lookback,
                      delay=delay,
                      min_index=0,
                      max_index=200000,
                      shuffle=True,
                      step=step, 
                      batch_size=batch_size)
val_gen = generator(float_data,
                    lookback=lookback,
                    delay=delay,
                    min_index=200001,
                    max_index=300000,
                    step=step,
                    batch_size=batch_size)
test_gen = generator(float_data,
                     lookback=lookback,
                     delay=delay,
                     min_index=300001,
                     max_index=None,
                     step=step,
                     batch_size=batch_size)

# This is how many steps to draw from `val_gen`
# in order to see the whole validation set:
val_steps = (300000 - 200001 - lookback) // batch_size

# This is how many steps to draw from `test_gen`
# in order to see the whole test set:
test_steps = (len(float_data) - 300001 - lookback) // batch_size

## A recurrent model


Instead of the `LSTM` layer introduced in the previous section, we will use the `GRU` layer, developed by Cho et al. in 2014. `GRU` layers 
(which stands for "gated recurrent unit") work by leveraging the same principle as LSTM, but they are somewhat streamlined and thus cheaper 
to run, albeit they may not have quite as much representational power as LSTM. This trade-off between computational expensiveness and 
representational power is seen everywhere in machine learning.

The model below uses recurrent dropout to help prevent overfitting.  

In [0]:
from keras.models import Sequential
from keras import layers, Input, Model
from keras.optimizers import RMSprop

input_tensor = Input(shape=(None, float_data.shape[-1]))
x = layers.GRU(16,
               dropout=0.2,
               recurrent_dropout=0.2)(input_tensor)
x = layers.Dense(1)(x)
y = layers.Lambda(lambda x: x[:,-1,1] )(input_tensor)
output_tensor = layers.add([x,y])
model = Model(input_tensor, output_tensor)
#model = Sequential()
#model.add(layers.GRU(32,
#                     dropout=0.2,
#                     recurrent_dropout=0.2,
#                     input_shape=(None, float_data.shape[-1])))
#model.add(layers.Dense(1))

model.compile(optimizer=RMSprop(), loss='mae')
history = model.fit_generator(train_gen,
                              steps_per_epoch=500,
                              epochs=40,
                              validation_data=val_gen,
                              validation_steps=val_steps)

Epoch 1/40
500/500 [==============================] - 81s 161ms/step - loss: 0.3126 - val_loss: 0.2763
Epoch 2/40
500/500 [==============================] - 77s 153ms/step - loss: 0.3003 - val_loss: 0.2738
Epoch 3/40
500/500 [==============================] - 77s 153ms/step - loss: 0.2952 - val_loss: 0.2726
Epoch 4/40
500/500 [==============================] - 77s 154ms/step - loss: 0.2933 - val_loss: 0.2705
Epoch 5/40
500/500 [==============================] - 76s 153ms/step - loss: 0.2927 - val_loss: 0.2699
Epoch 6/40
500/500 [==============================] - 77s 154ms/step - loss: 0.2908 - val_loss: 0.2677
Epoch 7/40
500/500 [==============================] - 77s 154ms/step - loss: 0.2889 - val_loss: 0.2683
Epoch 8/40
500/500 [==============================] - 77s 154ms/step - loss: 0.2881 - val_loss: 0.2698
Epoch 9/40
500/500 [==============================] - 77s 154ms/step - loss: 0.2863 - val_loss: 0.2656
Epoch 10/40
500/500 [==============================] - 77s 153ms/step - l

In [0]:
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(loss))

plt.figure()

plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()